In [2]:
import pandas as pd
import time

In [3]:
# Custom Exceptions and loggingsetup
class InvalidSharesError(Exception):
    pass
class InvalidPriceError(Exception):
    pass
import logging
logging.basicConfig(level=logging.INFO)

In [4]:
# Stock Class
class Stock:
    def __init__ (self, symbol, shares, purchase_price, current_price):
        if not isinstance (shares, (int,float)) or shares <=0:
            raise InvalidSharesError (f"Shares must be a positive number, got {shares}")
        if not isinstance (purchase_price, (int,float)) or purchase_price <=0:
            raise InvalidPriceError (f"Purchase Price must be a positive number, got {purchase_price}")
        if not isinstance (current_price, (int,float)) or current_price <=0:
            raise  InvalidPriceError (f"Current Price must be a positive number, got {current_price}")
        self.symbol=symbol
        self.shares=shares
        self.purchase_price=purchase_price
        self.current_price=current_price
        logging.info (f"Stock object created. Shares: {self.shares}, Symbol: {self.symbol}")

    @property
    def cost_basis(self):
        return self.purchase_price * self.shares

    @property
    def current_value(self):
        return self.current_price * self.shares

    @property
    def profit_loss(self):
        return self.current_value - self.cost_basis

    def __repr__ (self):
        return f"Symbol: {self.symbol}, Shares: {self.shares}, Purchase Price: {self.purchase_price}, Current Price: {self.current_price}" 

    def __str__ (self):
        return f"{self.shares} shares of {self.symbol} were purchased at {self.purchase_price} and are now worth {self.current_price} each"

    @classmethod
    def from_dict(cls,data):
        sym, shars, purch, curr=data['symbol'],data['shares'], data['purchase_price'], data['current_price']
        return cls(sym, shars, purch, curr)

    @staticmethod
    def validate_price (price):
        if not isinstance (price, (int,float)) or price <= 0:
            raise InvalidPriceError
        return True
        

In [6]:
# Timing decorator
def time_decorator(func):
        def wrapper (*args, **kwargs):
            start=time.time()
            result=func(*args,**kwargs)
            end=time.time()
            logging.info(f"{func.__name__}executed in {end-start:.4f} seconds")
            return result
        return wrapper

In [28]:
#Portfolio Class
class Portfolio:
    def __init__(self,name):
        self.name=name
        self.holdings=[]
        logging.info (f"{self.name} Portfolio object created.")
    def add_stock(self,stock):
        if not isinstance (stock,Stock):
            raise TypeError ("Expected a Stock object")
        self.holdings.append(stock)
    def remove_stock (self,symbol):
        initial_count=len(self.holdings)
        self.holdings=list(filter(lambda stock: stock.symbol!=symbol,self.holdings))
        end_count= len(self.holdings)
        if initial_count-end_count==0:
            raise ValueError(f"No Stock of symbol {symbol} exists in the Portfolio")
        logging.info (f"{symbol} Stock removed from {self.name} Portfolio")
    @time_decorator
    def total_value(self):
        return sum(stock.current_value for stock in self.holdings)
    def total_cost_basis(self):
        return sum(stock.cost_basis for stock in self.holdings)
    def total_profit_loss(self):
        return self.total_value()-self.total_cost_basis() 
    def get_holdings_summary(self):
        summary=[]
        for stock in self.holdings:
            summary.append({
                "symbol": stock.symbol,
                "shares": stock.shares,
                "current_value": stock.current_value,
                "profit_loss": stock.profit_loss
            })
        return summary

    def __repr__(self):
        return f"Name: {self.name}, Holdings: {len(self.holdings)}"
    def __len__(self):
        return len(self.holdings)
    def __getitem__(self,index):
        return self.holdings[index]

    def load_from_csv(self, filename):
        try:
            df = pd.read_csv(filename)
        except FileNotFoundError:
            logging.error(f"File not found: {filename}")
            raise
        for _, row in df.iterrows():
            stock_data={
                "symbol":row["symbol"],
                'shares': row['shares'],
                'purchase_price': row['purchase_price'],
                'current_price': row['current_price']
        }
            stock=Stock.from_dict(stock_data)
            self.add_stock(stock)
        logging.info(f"Loaded {len(df)} stocks from {filename}")

    def filter_stocks(self, min_value=0):
        filtered_stocks=list(filter(lambda s: s.current_value > min_value, self.holdings))
        symbols=list(map(lambda y: y.symbol, filtered_stocks))
        return filtered_stocks, symbols

    def display_holdings(self):
        for index,holding in enumerate(self.holdings, start=1):
            print(f"{index}. {holding.symbol}")



In [34]:
# Demonstartions
if __name__ == "__main__":
    # 1. Create a sample CSV file
    csv_content = """symbol,shares,purchase_price,current_price,sector
AAPL,10,150.0,175.0,Technology
MSFT,5,200.0,210.0,Technology
TSLA,2,600.0,650.0,Automotive
JPM,8,120.0,130.0,Finance
"""

    csv_filename = "sample_portfolio.csv"
    with open(csv_filename, "w") as f:
        f.write(csv_content)
    print(f"Created sample CSV: {csv_filename}")

    # 2. Create a portfolio and load data
    my_portfolio = Portfolio("Retirement Fund")
    my_portfolio.load_from_csv(csv_filename)

    # 3. Display holdings with enumerate
    my_portfolio.display_holdings()

    # 4. Filter stocks with value > $1000
    filtered, symbols = my_portfolio.filter_stocks(min_value=1000)
    print(f"\nStocks with current value > $1000: {symbols}")

    # 5. Show overall totals (with timing decorator)
    print(f"\nTotal portfolio value: ${my_portfolio.total_value():,.2f}")
    print(f"Total cost basis: ${my_portfolio.total_cost_basis():,.2f}")
    print(f"Total profit/loss: ${my_portfolio.total_profit_loss():,.2f}")

INFO:root:Retirement Fund Portfolio object created.
INFO:root:Stock object created. Shares: 10, Symbol: AAPL
INFO:root:Stock object created. Shares: 5, Symbol: MSFT
INFO:root:Stock object created. Shares: 2, Symbol: TSLA
INFO:root:Stock object created. Shares: 8, Symbol: JPM
INFO:root:Loaded 4 stocks from sample_portfolio.csv
INFO:root:total_valueexecuted in 0.0000 seconds
INFO:root:total_valueexecuted in 0.0000 seconds


Created sample CSV: sample_portfolio.csv
1. AAPL
2. MSFT
3. TSLA
4. JPM

Stocks with current value > $1000: ['AAPL', 'MSFT', 'TSLA', 'JPM']

Total portfolio value: $5,140.00
Total cost basis: $4,660.00
Total profit/loss: $480.00
